In [ ]:
import azureml
import json

from azureml.core.model import Model, InferenceConfig
from azureml.core import Workspace, Datastore, Experiment, Environment, ScriptRunConfig, Run 
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget

from azureml.widgets import RunDetails

from azureml.core.webservice import AciWebservice #Webservice, AksWebservice, AciWebservice
from azureml.core.conda_dependencies import CondaDependencies 

from azureml.train.hyperdrive import RandomParameterSampling
from azureml.train.hyperdrive import normal, uniform, choice
from azureml.train.hyperdrive import HyperDriveConfig
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, uniform, PrimaryMetricGoal

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

# Connect to Workspace

In [ ]:
from azureml.core.authentication import AzureCliAuthentication

cli_auth = AzureCliAuthentication(cloud="AzureCloud")

ws = Workspace(workspace_name="Cybertron",resource_group="Cybertron-RG",subscription_id="431dbae5-40ca-438a-8daf-77d7d5580b41",auth=cli_auth)
print("Workspace:",ws.name)

# Connect to compute for training
compute_target = ComputeTarget(workspace=ws, name="OptimusPrime")
print("Compute Target:",compute_target.name)

# Connect to the datastore for the training images
ds = Datastore.get_default(ws)
print("Datastore:",ds.name)

# Connect to the experiment
exp = Experiment(workspace=ws, name='Simpsons-PyTorch')
print("Experiment:",exp.name)
print("Workspace:",ws.name)

# Connect to compute for training
compute_target = ComputeTarget(workspace=ws, name="OptimusPrime")
print("Compute Target:",compute_target.name)

# Connect to the datastore for the training images
ds = Datastore.get_default(ws)
print("Datastore:",ds.name)

# Connect to the experiment
exp = Experiment(workspace=ws, name='Simpsons-PyTorch')
print("Experiment:",exp.name)

# Remote training

In [ ]:
simpsons_ds = Dataset.get_by_name(ws, name='lego-v1')
print("Dataset:",simpsons_ds.name,"Version:",simpsons_ds.version)

In [ ]:
curated_env_name = 'AzureML-pytorch-1.10-ubuntu18.04-py38-cuda11-gpu'
pytorch_env = Environment.get(workspace=ws, name=curated_env_name)
pytorch_env = pytorch_env.clone(new_name='pytorch-simpson-training')
print("Environment:",pytorch_env.name)

In [ ]:
args = [
    '--data-folder', simpsons_ds.as_named_input('simpsons').as_mount(),
    '--num-epochs', 10
]

config = ScriptRunConfig(
    source_directory = '../scripts/train', 
    script = 'train.py', 
    compute_target = compute_target,
    environment = pytorch_env,
    arguments=args,
)

In [ ]:
run = exp.submit(config)
print("Run:",run.id)

In [ ]:
RunDetails(run).show()

In [ ]:
model = run.register_model(
   model_name='Simpsons-Classification',
   model_path='outputs',
   model_framework='PyTorch',
   model_framework_version='1.6',
   description="Lego figures PyTorch Classifier",
   tags={'Conference':''},
)

print("Model '{}' version {} registered ".format(model.name,model.version))

# Test deployment to an Azure Container Instance

In [ ]:
myenv = Environment(name="simpsons-inference")

conda_dep = CondaDependencies()

# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("torch")
conda_dep.add_pip_package("torchvision")
conda_dep.add_pip_package("pillow==5.4.1")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

In [ ]:
inference_config = InferenceConfig(
    entry_script = "../scripts/score/score.py", 
    environment = myenv
)

In [ ]:
deploy_config = AciWebservice.deploy_configuration(
    cpu_cores = 2, 
    memory_gb = 4,
    description='Simpson Lego Classifier'
)

In [ ]:
aci_service = Model.deploy(ws, 
    name="simpsons-classification-test", 
    models = [model], 
    inference_config = inference_config, 
    deployment_config = deploy_config, 
    overwrite = True
)

aci_service.wait_for_deployment(show_output=True)

In [ ]:
img_name = "https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Bart.jpg"
result = aci_service.run(input_data=json.dumps({ "image": img_name}))

print(result)

# Optimize the model

In [ ]:
param_sampling = RandomParameterSampling( {
        'num-epochs': choice(range(3,5)),
        'learning_rate': uniform(0.001, 0.005),
        'momentum': uniform(0.9, 0.99)
    }
)

args = [
    '--data-folder', simpsons_ds.as_named_input('simpsons').as_mount(),
]

script_run_config = ScriptRunConfig(
    source_directory = "../scripts/train", 
    script = 'train.py', 
    compute_target=compute_target,
    environment = pytorch_env,
    arguments=args,
)

hd_config = HyperDriveConfig(run_config = script_run_config,
                             hyperparameter_sampling = param_sampling,
                             primary_metric_name="accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=4,
                             max_concurrent_runs=4)

In [ ]:
hyperdrive_run = exp.submit(hd_config)

In [ ]:
RunDetails(hyperdrive_run).show()

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['accuracy'])
print('Learning rate:', parameter_values[3])
print('Momentum:', parameter_values[5])

In [ ]:
model = best_run.register_model(
    model_name='Simpsons-Classification',
    model_path='outputs',
    model_framework='PyTorch',
    model_framework_version='1.6',
    description="Simpsons best model",
    tags={'Conference':'Test 1','HPT':'true'},
)

print("Model '{}' version {} registered ".format(model.name,model.version))